In [1]:
import torchvision.datasets as datasets
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device :', device)

Using device : cpu


In [3]:
my_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))])


trainset = datasets.MNIST(root='./data', train=True,
                                        download=True, transform = my_transform)

subset = list(range(0, 4000, 1))

trainset_1 = torch.utils.data.Subset(trainset, subset)

trainloader = torch.utils.data.DataLoader(trainset_1, batch_size=128,
                                          shuffle=True, num_workers=4)

testset = datasets.MNIST(root='./data', train=False,
                                       download=True, transform = my_transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=4)
                          
print('Load MINST success')

Load MINST success


In [4]:
class Net(torch.nn.Module):
    def __init__(self, hidden_units):
        super(Net, self).__init__()
        self.layer1 = torch.nn.Linear(28*28, hidden_units)
        #self.layer2 = torch.nn.Linear(hidden_units, hidden_units)
        self.layer3 = torch.nn.Linear(hidden_units, 10)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.relu(self.layer1(x))
        #x = torch.relu(self.layer2(x))
        x = torch.sigmoid(self.layer3(x))
        return x

In [5]:
class simple_FC(nn.Module):
    def __init__(self, n_hidden):
        super(simple_FC, self).__init__()
        self.features = nn.Sequential(
            nn.Flatten(),
            nn.Linear(784, n_hidden),
            nn.ReLU()
        )
        self.classifier = nn.Linear(n_hidden, 10)
        
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

In [6]:
def train_and_evaluate_model(model, hidden_unit, optimizer, criterion, n_epochs):
    train_losses, test_losses = [], []
    train_acc, test_acc, epoch = 0, 0, 0
    
    while epoch <= n_epochs and train_acc < 0.99:
        if epoch >= 1:
            print("Epoch : %d ; Train Loss : %f ; Train Acc : %.3f ; Test Loss : %f ; Test Acc : %.3f ; LR : %.3f" 
                  % (epoch, train_losses[-1], train_acc, test_losses[-1], test_acc, optimizer.param_groups[0]['lr']))
        epoch += 1
        
        if hidden_unit <= 50 and epoch % 50 == 1:
            optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr'] * 0.9
    
        model.train()
        
        train_loss, correct, total = 0.0, 0, 0
        for inputs, labels in trainloader:
            labels = torch.nn.functional.one_hot(labels, num_classes=10).float()
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels.argmax(1)).sum().item()

        train_losses.append(train_loss/len(trainloader))
        train_acc = correct/total
        
        
        model.eval()
        
        test_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in testloader:
                labels = torch.nn.functional.one_hot(labels, num_classes=10).float()
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                test_loss += loss.item()
                
                _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels.argmax(1)).sum().item()
            
        test_losses.append(test_loss/len(testloader))
        test_acc = correct/total

    return train_losses[-1], train_acc, test_losses[-1], test_acc

In [7]:
# Training Settings

hidden_units = [1, 5, 10, 20, 30, 40, 45, 47, 49, 50, 51, 53, 55, 60, 80, 100]
n_epochs = 600
momentum = 0.95
learning_rate = 0.05

parameters = []
for hidden_unit in hidden_units:
    parameters.append(784 * hidden_unit + hidden_unit * 10)

In [ ]:
train_losses, train_accs, test_losses, test_accs = [], [], [], []

for hidden_unit in hidden_units:
    model = simple_FC(hidden_unit)
    
    if hidden_unit == 1:
        torch.nn.init.xavier_uniform_(model.features[1].weight, gain=1.0)
        torch.nn.init.xavier_uniform_(model.classifier.weight, gain=1.0)
    else: 
        torch.nn.init.normal_(model.features[1].weight, mean=0.0, std=0.1)
        torch.nn.init.normal_(model.classifier.weight, mean=0.0, std=0.1)
    
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(), momentum=momentum, lr=learning_rate)
    criterion = torch.nn.MSELoss()
    
    train_loss, train_acc, test_loss, test_acc = train_and_evaluate_model(model, hidden_unit, optimizer, criterion, n_epochs)
    print("\nHidden Neurons : %d ; Train Loss : %f ; Train Acc : %.3f ; Test Loss : %f ; Test Acc : %.3f\n\n" \
              % (hidden_unit, train_loss, train_acc, test_loss, test_acc))
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)

Epoch : 1 ; Train Loss : 0.221349 ; Train Acc : 0.109 ; Test Loss : 0.158171 ; Test Acc : 0.125 ; LR : 0.045
Epoch : 2 ; Train Loss : 0.115999 ; Train Acc : 0.090 ; Test Loss : 0.101261 ; Test Acc : 0.062 ; LR : 0.045
Epoch : 3 ; Train Loss : 0.095103 ; Train Acc : 0.100 ; Test Loss : 0.091455 ; Test Acc : 0.125 ; LR : 0.045
Epoch : 4 ; Train Loss : 0.090915 ; Train Acc : 0.114 ; Test Loss : 0.090256 ; Test Acc : 0.062 ; LR : 0.045
Epoch : 5 ; Train Loss : 0.090185 ; Train Acc : 0.108 ; Test Loss : 0.090000 ; Test Acc : 0.125 ; LR : 0.045
Epoch : 6 ; Train Loss : 0.090060 ; Train Acc : 0.108 ; Test Loss : 0.089997 ; Test Acc : 0.125 ; LR : 0.045
Epoch : 7 ; Train Loss : 0.090045 ; Train Acc : 0.094 ; Test Loss : 0.089987 ; Test Acc : 0.125 ; LR : 0.045
Epoch : 8 ; Train Loss : 0.090019 ; Train Acc : 0.105 ; Test Loss : 0.090008 ; Test Acc : 0.125 ; LR : 0.045
Epoch : 9 ; Train Loss : 0.090021 ; Train Acc : 0.111 ; Test Loss : 0.089985 ; Test Acc : 0.125 ; LR : 0.045
Epoch : 10 ; Train 

Epoch : 76 ; Train Loss : 0.090044 ; Train Acc : 0.111 ; Test Loss : 0.090005 ; Test Acc : 0.125 ; LR : 0.041
Epoch : 77 ; Train Loss : 0.090003 ; Train Acc : 0.102 ; Test Loss : 0.090002 ; Test Acc : 0.125 ; LR : 0.041
Epoch : 78 ; Train Loss : 0.090050 ; Train Acc : 0.107 ; Test Loss : 0.089989 ; Test Acc : 0.125 ; LR : 0.041
Epoch : 79 ; Train Loss : 0.090011 ; Train Acc : 0.111 ; Test Loss : 0.090003 ; Test Acc : 0.125 ; LR : 0.041
Epoch : 80 ; Train Loss : 0.089987 ; Train Acc : 0.105 ; Test Loss : 0.089987 ; Test Acc : 0.062 ; LR : 0.041
Epoch : 81 ; Train Loss : 0.090009 ; Train Acc : 0.106 ; Test Loss : 0.089989 ; Test Acc : 0.125 ; LR : 0.041
Epoch : 82 ; Train Loss : 0.090055 ; Train Acc : 0.111 ; Test Loss : 0.089988 ; Test Acc : 0.125 ; LR : 0.041
Epoch : 83 ; Train Loss : 0.090008 ; Train Acc : 0.109 ; Test Loss : 0.089997 ; Test Acc : 0.062 ; LR : 0.041
Epoch : 84 ; Train Loss : 0.089999 ; Train Acc : 0.111 ; Test Loss : 0.089989 ; Test Acc : 0.125 ; LR : 0.041
Epoch : 85

Epoch : 151 ; Train Loss : 0.089998 ; Train Acc : 0.108 ; Test Loss : 0.089984 ; Test Acc : 0.062 ; LR : 0.033
Epoch : 152 ; Train Loss : 0.089996 ; Train Acc : 0.111 ; Test Loss : 0.090001 ; Test Acc : 0.125 ; LR : 0.033
Epoch : 153 ; Train Loss : 0.090017 ; Train Acc : 0.095 ; Test Loss : 0.089984 ; Test Acc : 0.125 ; LR : 0.033
Epoch : 154 ; Train Loss : 0.089985 ; Train Acc : 0.103 ; Test Loss : 0.089999 ; Test Acc : 0.125 ; LR : 0.033
Epoch : 155 ; Train Loss : 0.090027 ; Train Acc : 0.106 ; Test Loss : 0.089998 ; Test Acc : 0.125 ; LR : 0.033
Epoch : 156 ; Train Loss : 0.090004 ; Train Acc : 0.111 ; Test Loss : 0.089995 ; Test Acc : 0.125 ; LR : 0.033
Epoch : 157 ; Train Loss : 0.089984 ; Train Acc : 0.103 ; Test Loss : 0.089999 ; Test Acc : 0.062 ; LR : 0.033
Epoch : 158 ; Train Loss : 0.089998 ; Train Acc : 0.107 ; Test Loss : 0.089998 ; Test Acc : 0.125 ; LR : 0.033
Epoch : 159 ; Train Loss : 0.089996 ; Train Acc : 0.107 ; Test Loss : 0.090000 ; Test Acc : 0.125 ; LR : 0.033
E

Epoch : 225 ; Train Loss : 0.089983 ; Train Acc : 0.112 ; Test Loss : 0.089993 ; Test Acc : 0.125 ; LR : 0.030
Epoch : 226 ; Train Loss : 0.090015 ; Train Acc : 0.102 ; Test Loss : 0.090002 ; Test Acc : 0.125 ; LR : 0.030
Epoch : 227 ; Train Loss : 0.089993 ; Train Acc : 0.107 ; Test Loss : 0.089997 ; Test Acc : 0.125 ; LR : 0.030
Epoch : 228 ; Train Loss : 0.089990 ; Train Acc : 0.111 ; Test Loss : 0.089991 ; Test Acc : 0.125 ; LR : 0.030
Epoch : 229 ; Train Loss : 0.089987 ; Train Acc : 0.107 ; Test Loss : 0.090002 ; Test Acc : 0.062 ; LR : 0.030
Epoch : 230 ; Train Loss : 0.090001 ; Train Acc : 0.109 ; Test Loss : 0.089995 ; Test Acc : 0.125 ; LR : 0.030
Epoch : 231 ; Train Loss : 0.090011 ; Train Acc : 0.096 ; Test Loss : 0.089988 ; Test Acc : 0.062 ; LR : 0.030
Epoch : 232 ; Train Loss : 0.089997 ; Train Acc : 0.108 ; Test Loss : 0.090008 ; Test Acc : 0.062 ; LR : 0.030
Epoch : 233 ; Train Loss : 0.089997 ; Train Acc : 0.109 ; Test Loss : 0.089988 ; Test Acc : 0.125 ; LR : 0.030
E

Epoch : 299 ; Train Loss : 0.089977 ; Train Acc : 0.111 ; Test Loss : 0.089980 ; Test Acc : 0.125 ; LR : 0.027
Epoch : 300 ; Train Loss : 0.089979 ; Train Acc : 0.108 ; Test Loss : 0.090006 ; Test Acc : 0.062 ; LR : 0.027
Epoch : 301 ; Train Loss : 0.089974 ; Train Acc : 0.104 ; Test Loss : 0.089989 ; Test Acc : 0.125 ; LR : 0.024
Epoch : 302 ; Train Loss : 0.089970 ; Train Acc : 0.108 ; Test Loss : 0.089984 ; Test Acc : 0.125 ; LR : 0.024
Epoch : 303 ; Train Loss : 0.089989 ; Train Acc : 0.104 ; Test Loss : 0.089993 ; Test Acc : 0.125 ; LR : 0.024
Epoch : 304 ; Train Loss : 0.089989 ; Train Acc : 0.104 ; Test Loss : 0.089986 ; Test Acc : 0.125 ; LR : 0.024
Epoch : 305 ; Train Loss : 0.089973 ; Train Acc : 0.111 ; Test Loss : 0.089987 ; Test Acc : 0.125 ; LR : 0.024
Epoch : 306 ; Train Loss : 0.089987 ; Train Acc : 0.098 ; Test Loss : 0.089998 ; Test Acc : 0.125 ; LR : 0.024
Epoch : 307 ; Train Loss : 0.089996 ; Train Acc : 0.105 ; Test Loss : 0.089992 ; Test Acc : 0.125 ; LR : 0.024
E

...

Epoch : 243 ; Train Loss : 0.052992 ; Train Acc : 0.502 ; Test Loss : 0.057326 ; Test Acc : 0.438 ; LR : 0.030
Epoch : 244 ; Train Loss : 0.053114 ; Train Acc : 0.503 ; Test Loss : 0.057751 ; Test Acc : 0.438 ; LR : 0.030
Epoch : 245 ; Train Loss : 0.053049 ; Train Acc : 0.502 ; Test Loss : 0.057240 ; Test Acc : 0.438 ; LR : 0.030
Epoch : 246 ; Train Loss : 0.053048 ; Train Acc : 0.502 ; Test Loss : 0.057318 ; Test Acc : 0.438 ; LR : 0.030
Epoch : 247 ; Train Loss : 0.052652 ; Train Acc : 0.492 ; Test Loss : 0.057255 ; Test Acc : 0.438 ; LR : 0.030
Epoch : 248 ; Train Loss : 0.052885 ; Train Acc : 0.503 ; Test Loss : 0.057329 ; Test Acc : 0.438 ; LR : 0.030
Epoch : 249 ; Train Loss : 0.053296 ; Train Acc : 0.504 ; Test Loss : 0.057461 ; Test Acc : 0.438 ; LR : 0.030
Epoch : 250 ; Train Loss : 0.053009 ; Train Acc : 0.501 ; Test Loss : 0.057266 ; Test Acc : 0.438 ; LR : 0.030
Epoch : 251 ; Train Loss : 0.052770 ; Train Acc : 0.503 ; Test Loss : 0.057227 ; Test Acc : 0.438 ; LR : 0.027
E

Epoch : 317 ; Train Loss : 0.052510 ; Train Acc : 0.503 ; Test Loss : 0.057720 ; Test Acc : 0.438 ; LR : 0.024
Epoch : 318 ; Train Loss : 0.052547 ; Train Acc : 0.494 ; Test Loss : 0.057619 ; Test Acc : 0.438 ; LR : 0.024
Epoch : 319 ; Train Loss : 0.052956 ; Train Acc : 0.503 ; Test Loss : 0.057638 ; Test Acc : 0.438 ; LR : 0.024
Epoch : 320 ; Train Loss : 0.052598 ; Train Acc : 0.504 ; Test Loss : 0.057860 ; Test Acc : 0.438 ; LR : 0.024
Epoch : 321 ; Train Loss : 0.052512 ; Train Acc : 0.503 ; Test Loss : 0.057580 ; Test Acc : 0.438 ; LR : 0.024
Epoch : 322 ; Train Loss : 0.053023 ; Train Acc : 0.503 ; Test Loss : 0.057612 ; Test Acc : 0.438 ; LR : 0.024
Epoch : 323 ; Train Loss : 0.052633 ; Train Acc : 0.504 ; Test Loss : 0.057757 ; Test Acc : 0.438 ; LR : 0.024
Epoch : 324 ; Train Loss : 0.052437 ; Train Acc : 0.504 ; Test Loss : 0.057669 ; Test Acc : 0.438 ; LR : 0.024
Epoch : 325 ; Train Loss : 0.052541 ; Train Acc : 0.505 ; Test Loss : 0.057930 ; Test Acc : 0.438 ; LR : 0.024
E

Epoch : 391 ; Train Loss : 0.052128 ; Train Acc : 0.505 ; Test Loss : 0.058027 ; Test Acc : 0.438 ; LR : 0.022
Epoch : 392 ; Train Loss : 0.052172 ; Train Acc : 0.504 ; Test Loss : 0.057952 ; Test Acc : 0.438 ; LR : 0.022
Epoch : 393 ; Train Loss : 0.052574 ; Train Acc : 0.505 ; Test Loss : 0.058200 ; Test Acc : 0.438 ; LR : 0.022
Epoch : 394 ; Train Loss : 0.052306 ; Train Acc : 0.505 ; Test Loss : 0.057935 ; Test Acc : 0.438 ; LR : 0.022
Epoch : 395 ; Train Loss : 0.052525 ; Train Acc : 0.504 ; Test Loss : 0.058012 ; Test Acc : 0.438 ; LR : 0.022
Epoch : 396 ; Train Loss : 0.052319 ; Train Acc : 0.505 ; Test Loss : 0.058104 ; Test Acc : 0.438 ; LR : 0.022
Epoch : 397 ; Train Loss : 0.052675 ; Train Acc : 0.504 ; Test Loss : 0.057972 ; Test Acc : 0.438 ; LR : 0.022
Epoch : 398 ; Train Loss : 0.052279 ; Train Acc : 0.505 ; Test Loss : 0.057950 ; Test Acc : 0.438 ; LR : 0.022
Epoch : 399 ; Train Loss : 0.052090 ; Train Acc : 0.504 ; Test Loss : 0.058107 ; Test Acc : 0.438 ; LR : 0.022
E

Epoch : 465 ; Train Loss : 0.052224 ; Train Acc : 0.505 ; Test Loss : 0.058440 ; Test Acc : 0.438 ; LR : 0.017
Epoch : 466 ; Train Loss : 0.052048 ; Train Acc : 0.506 ; Test Loss : 0.058313 ; Test Acc : 0.438 ; LR : 0.017
Epoch : 467 ; Train Loss : 0.052156 ; Train Acc : 0.506 ; Test Loss : 0.058450 ; Test Acc : 0.438 ; LR : 0.017
Epoch : 468 ; Train Loss : 0.052198 ; Train Acc : 0.505 ; Test Loss : 0.058261 ; Test Acc : 0.438 ; LR : 0.017
Epoch : 469 ; Train Loss : 0.051848 ; Train Acc : 0.506 ; Test Loss : 0.058314 ; Test Acc : 0.438 ; LR : 0.017
Epoch : 470 ; Train Loss : 0.052123 ; Train Acc : 0.505 ; Test Loss : 0.058321 ; Test Acc : 0.438 ; LR : 0.017
Epoch : 471 ; Train Loss : 0.051643 ; Train Acc : 0.506 ; Test Loss : 0.058508 ; Test Acc : 0.438 ; LR : 0.017
Epoch : 472 ; Train Loss : 0.052275 ; Train Acc : 0.506 ; Test Loss : 0.058359 ; Test Acc : 0.438 ; LR : 0.017
Epoch : 473 ; Train Loss : 0.052370 ; Train Acc : 0.507 ; Test Loss : 0.058291 ; Test Acc : 0.438 ; LR : 0.017
E

Epoch : 539 ; Train Loss : 0.052102 ; Train Acc : 0.506 ; Test Loss : 0.058585 ; Test Acc : 0.438 ; LR : 0.016
Epoch : 540 ; Train Loss : 0.051953 ; Train Acc : 0.506 ; Test Loss : 0.058572 ; Test Acc : 0.438 ; LR : 0.016
Epoch : 541 ; Train Loss : 0.052048 ; Train Acc : 0.506 ; Test Loss : 0.058564 ; Test Acc : 0.438 ; LR : 0.016
Epoch : 542 ; Train Loss : 0.052147 ; Train Acc : 0.506 ; Test Loss : 0.058696 ; Test Acc : 0.438 ; LR : 0.016
Epoch : 543 ; Train Loss : 0.051921 ; Train Acc : 0.506 ; Test Loss : 0.058595 ; Test Acc : 0.438 ; LR : 0.016
Epoch : 544 ; Train Loss : 0.052081 ; Train Acc : 0.507 ; Test Loss : 0.058585 ; Test Acc : 0.438 ; LR : 0.016
Epoch : 545 ; Train Loss : 0.052351 ; Train Acc : 0.507 ; Test Loss : 0.058560 ; Test Acc : 0.438 ; LR : 0.016
Epoch : 546 ; Train Loss : 0.051857 ; Train Acc : 0.506 ; Test Loss : 0.058611 ; Test Acc : 0.438 ; LR : 0.016
Epoch : 547 ; Train Loss : 0.052235 ; Train Acc : 0.506 ; Test Loss : 0.058671 ; Test Acc : 0.438 ; LR : 0.016
E

In [ ]:
plt.plot(hidden_units, train_losses, marker='o', label='train')
plt.plot(hidden_units, test_losses, marker='o', label='test')
plt.xlabel('Number of Hidden Units')
plt.ylabel('MSE loss')
plt.title('Double Descent Curve')
plt.show()

In [ ]:
plt.plot(parameters, train_losses, marker='o', label='train')
plt.plot(parameters, test_losses, marker='o', label='test')
plt.xlabel('Number of Model Parameters')
plt.ylabel('MSE loss')
plt.title('Double Descent Curve')
plt.show()

In [ ]:
plt.plot(hidden_units, train_accs, marker='o', label='train')
plt.plot(hidden_units, test_accs, marker='o', label='test')
plt.xlabel('Number of Hidden Units')
plt.ylabel('Accuracy')
plt.title('Double Descent Curve')
plt.show()

In [ ]:
plt.plot(parameters, train_accs, marker='o', label='train')
plt.plot(parameters, test_accs, marker='o', label='test')
plt.xlabel('Number of Model Parameters')
plt.ylabel('MSE loss')
plt.title('Double Descent Curve')
plt.show()